<a href="https://colab.research.google.com/github/at1609/Distracted-Driver-MultiAction-Classification/blob/main/Astech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python==4.5.1.48

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 MB 13.4 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.7.0.72
    Uninstalling opencv-python-4.7.0.72:
      Successfully uninstalled opencv-python-4.7.0.72


In [ ]:
#from keras.applications.vgg16 import VGG16

In [ ]:
#cnn_model = VGG16(weights='imagenet')

553467096/553467096 [==============================] - 4s 0us/step


In [ ]:
#cnn_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [1]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input

# Define the input shape
input_shape = (64, 64, 3)

# Create the VGG16 model
model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)


58889256/58889256 [==============================] - 0s 0us/step


In [2]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

In [3]:
import cv2 
import os

input_dir = "/content/drive/MyDrive/furniture_images"
output_dir = "/content/resized"

if not os.path.exists('output_dir'):
  os.makedirs(output_dir)

for file_name in os.listdir(input_dir):

  if file_name.endswith('jpg'):

    img = cv2.imread(os.path.join(input_dir, file_name))

    resized_img = cv2.resize(img, (64,64))

    cv2.imwrite(os.path.join(output_dir, file_name), resized_img)



In [4]:
import cv2
import numpy as np
import time
from sklearn.model_selection import train_test_split
import os

# Define the encoder-decoder model to compress the features
from keras.layers import Input, Dense, Dropout, Flatten, Reshape, BatchNormalization, LSTM, Conv2D, MaxPooling2D, Conv2DTranspose, UpSampling2D
from keras.models import Model, Sequential
from sklearn.model_selection import train_test_split 

input_shape = (2,2,512)
# Define the shapeencoder model
latent_dim = 64

# Define the encoder model
input_img = Input(shape=input_shape)
x = Flatten()(input_img)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
encoded = Dense(64, activation='relu')(x)

encoder_model = Model(input_img, encoded)

# Define the decoder model
decoder_input = Input(shape=(latent_dim,))
x = Dense(128, activation='relu')(decoder_input)
x = Dense(256, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(2048, activation='relu')(x)
x = Reshape((2, 2, 512))(x)
decoded_output = x

decoder_model = Model(decoder_input, decoded_output)


#decoder_model = Model(decoded_input, decoded_output)

# Define the autoencoder model
autoencoder = Model(input_img, decoder_model(encoder_model(input_img)))

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='mse')

# Compile the model
#autoencoder.compile(loss='binary_crossentropy', optimizer='adam')

folder_path = '/content/resized'

images = [f for f in os.listdir(folder_path) if f.endswith(".jpg")]

train, test = train_test_split(images, test_size = 0.3, random_state = 42)
train_data, valid_data = train_test_split(train, test_size=0.3, random_state = 42)

x_train, y_train = train_test_split(train_data, test_size = 0.2 , random_state = 42)
x_valid, y_valid = train_test_split(valid_data, test_size = 0.2, random_state = 42)

x_test, y_test = train_test_split(test, test_size = 0.2 , random_state = 42)

In [5]:
from keras.applications.vgg16 import preprocess_input
# Assuming x_train is a 3D tensor with shape (num_samples, height, width)
x_train = np.random.rand(64, 64, 3)
y_train = np.random.rand(64,64,3)
x_valid = np.random.rand(64, 64, 3)
y_valid = np.random.rand(64, 64, 3)
x_test = np.random.rand(64, 64, 3)
y_test = np.random.rand(64, 64, 3)


# Add batch_size dimension
x_train = np.expand_dims(x_train, axis=0)
y_train = np.expand_dims(y_train, axis=0)
x_valid = np.expand_dims(x_valid, axis=0)
y_valid = np.expand_dims(y_valid, axis=0)
x_test = np.expand_dims(x_test, axis=0)
y_test= np.expand_dims(y_test, axis=0)

# Preprocess input data
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)
x_valid = preprocess_input(x_valid)
y_valid = preprocess_input(y_valid)
x_test = preprocess_input(x_test)
y_test = preprocess_input(y_test)

In [9]:
os.makedirs('/content/cnn_features.npy')
        

In [21]:
from google.colab.patches import cv2_imshow
x_train_features = model.predict(x_train)
y_train_features = model.predict(y_train)
x_val_features = model.predict(x_valid)
y_val_features = model.predict(y_valid)
x_test_features = model.predict(x_test)
y_test_features = model.predict(y_test)

#train_rnn_features = rnn_model.predict(train_data)
#val_rnn_features = rnn_model.predict(val_data)
#test_rnn_features = rnn_model.predict(test_data)

# Train the model
autoencoder.fit(x_train_features, y_train_features, validation_data=(x_val_features, y_val_features), epochs=10, batch_size=64)

# Evaluate the model on the testing set
autoencoder.evaluate(x_test_features, y_test_features)

# Save the model to disk
autoencoder.save("autoencoder.h5")

# Define a function to capture and compress frames
def capture_and_compress_video():
    # Initialize the camera
    camera = cv2.VideoCapture(0)
    
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output.avi',fourcc, 20.0, (64,64))
    
    # Loop over the frames
    while(camera.isOpened()):
        # Capture the frame
        ret, frame = camera.read()
        
        # Break the loop if no frame is captured
        if not ret:
            break
        

1/1 [==============================] - 0s 72ms/step
Epoch 1/10
1/1 [==============================] - 0s 136ms/step - loss: 0.0372 - val_loss: 16.4767
Epoch 2/10
1/1 [==============================] - 0s 95ms/step - loss: 0.0372 - val_loss: 16.4767
Epoch 3/10
1/1 [==============================] - 0s 106ms/step - loss: 0.0372 - val_loss: 16.4767
Epoch 4/10
1/1 [==============================] - 0s 120ms/step - loss: 0.0372 - val_loss: 16.4767
Epoch 5/10
1/1 [==============================] - 0s 118ms/step - loss: 0.0372 - val_loss: 16.4766
Epoch 6/10
1/1 [==============================] - 0s 109ms/step - loss: 0.0372 - val_loss: 16.4766
Epoch 7/10
1/1 [==============================] - 0s 101ms/step - loss: 0.0372 - val_loss: 16.4767
Epoch 8/10
1/1 [==============================] - 0s 125ms/step - loss: 0.0372 - val_loss: 16.4767
Epoch 9/10
1/1 [==============================] - 0s 107ms/step - loss: 0.0372 - val_loss: 16.4767
Epoch 10/10
1/1 [==============================] - 0s 39ms

In [22]:
from google.colab.patches import cv2_imshow
x_train_features = model.predict(x_train)
y_train_features = model.predict(y_train)
x_val_features = model.predict(x_valid)
y_val_features = model.predict(y_valid)
x_test_features = model.predict(x_test)
y_test_features = model.predict(y_test)

#train_rnn_features = rnn_model.predict(train_data)
#val_rnn_features = rnn_model.predict(val_data)
#test_rnn_features = rnn_model.predict(test_data)

# Train the model
autoencoder.fit(x_train_features, y_train_features, validation_data=(x_val_features, y_val_features), epochs=10, batch_size=64)

# Evaluate the model on the testing set
autoencoder.evaluate(x_test_features, y_test_features)

# Save the model to disk
autoencoder.save("autoencoder.h5")

# Define a function to capture and compress frames
def capture_and_compress_video():
    # Initialize the camera
    camera = cv2.VideoCapture(0)
    
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output.avi',fourcc, 20.0, (64,64))
    
    # Loop over the frames
    while(camera.isOpened()):
        # Capture the frame
        ret, frame = camera.read()
        
        # Break the loop if no frame is captured
        if not ret:
            break
        
        # Preprocess the video frame as required
        # Convert the frame to the input shape required by the model
                # Resize the frame to the input shape required by the model
        input_frame = cv2.resize(frame, input_shape[:2])
        
        # Normalize the pixel values of the frame
        input_frame = input_frame.astype('float32') / 255.0
        
        # Extract the required features from the frame using the pre-trained CNN and RNN models
        cnn_features = model.predict(np.array([input_frame]))
        #rnn_features = rnn_model.predict(np.array([input_frame]))
        
        # Compress the features using the encoder model
        compressed_cnn_features = encoder.predict(cnn_features)
        #compressed_rnn_features = encoder.predict(rnn_features)
        
        # Save the compressed features to disk
        # Mount Google Drive to Colab
        from google.colab import drive
        drive.mount('/content/drive')

        # Save the compressed features to a file in Google Drive
        np.save('/content/cnn_features.npy', compressed_cnn_features)

        #np.save("rnn_features.npy", compressed_rnn_features)
        
        # Write the frame to the output video
        out.write(frame)
        
        # Display the frame
        cv2_imshow('frame',frame)
        
        # Break the loop if the 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release the camera and the video writer
    camera.release()
    out.release()
    cv2.destroyAllWindows()
    
# Define a function to browse the recorded content
def browse_recorded_content():
    # Load the compressed features from disk
    compressed_cnn_features = np.load('/content/cnn_features.npy')
    #compressed_rnn_features = np.load("rnn_features.npy")
    
    # Decompress the features using the decoder model
    decompressed_cnn_features = decoder.predict(compressed_cnn_features)
    #decompressed_rnn_features = decoder.predict(compressed_rnn_features)
    
    # Reconstruct the video frames from the decompressed features
    reconstructed_frames = []
    for i in range(decompressed_cnn_features.shape[0]):
        reconstructed_frame = cv2.resize(decompressed_cnn_features[i], (224, 224))
        reconstructed_frames.append(reconstructed_frame)
    
    # Play the reconstructed video
    for frame in reconstructed_frames:
        cv2_imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release the window
    cv2.destroyAllWindows()
    
# Call the functions to capture and compress the video frames, and browse the recorded content
capture_and_compress_video()
browse_recorded_content()

1/1 [==============================] - 0s 73ms/step
Epoch 1/10
1/1 [==============================] - 0s 108ms/step - loss: 0.0372 - val_loss: 16.4767
Epoch 2/10
1/1 [==============================] - 0s 72ms/step - loss: 0.0372 - val_loss: 16.4767
Epoch 3/10
1/1 [==============================] - 0s 67ms/step - loss: 0.0372 - val_loss: 16.4767
Epoch 4/10
1/1 [==============================] - 0s 86ms/step - loss: 0.0372 - val_loss: 16.4767
Epoch 5/10
1/1 [==============================] - 0s 73ms/step - loss: 0.0372 - val_loss: 16.4767
Epoch 6/10
1/1 [==============================] - 0s 72ms/step - loss: 0.0372 - val_loss: 16.4767
Epoch 7/10
1/1 [==============================] - 0s 76ms/step - loss: 0.0372 - val_loss: 16.4767
Epoch 8/10
1/1 [==============================] - 0s 73ms/step - loss: 0.0372 - val_loss: 16.4767
Epoch 9/10
1/1 [==============================] - 0s 87ms/step - loss: 0.0372 - val_loss: 16.4767
Epoch 10/10
1/1 [==============================] - 0s 34ms/step -

IsADirectoryError: ignored